# テキスト操作の比較
## ライブラリ準備

In [ ]:
import logging
import time
import json
import os

from dotenv import load_dotenv
import requests

import cohere
import anthropic
import openai
from openai import OpenAI

In [ ]:
load_dotenv()

CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 各サービスへのクライアントの定義

In [ ]:
translation_query = lambda text: f"""以下の英語テキストを日本語に翻訳してください。
翻訳は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
英語テキスト:
{text}

日本語翻訳:"""

summarization_query = lambda text: f"""以下のテキストを日本語で 400 字程度に要約してください。
要約は自然で流暢な日本語にし、原文のニュアンスと意味を正確に保ってください。
   
オリジナルテキスト:
{text}

日本語要約:"""

class AIModelClient:

    def _query(self, func: function, text: str, model: str = None) -> Dict:
        start_time = time.time()
        try:
            translation, model = func(text, model)
            elapsed_time = time.time() - start_time
            
            return {
                "result": translation,
                "time": elapsed_time,
                "model": model,
                "success": True,
                "error": None
            }
        except Exception as e:
            return {
                "result": None,
                "time": time.time() - start_time,
                "model": model,
                "success": False,
                "error": str(e)
            }

    def translate(self, text: str, model: str = None) -> Dict:
        return self._query(self._concrete_translate, text, model)

    def summarize(self, text: str, model: str = None) -> Dict:
        return self._query(self._concrete_summarize, text, model)
        start_time = time.time()

class ClaudeClient(AIModelClient):
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)

    def _concrete_translate(self, text: str, model: str) -> Dict:
        if model is None:
            model = "claude-sonnet-4-5"
        message = self.client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ]
        )        
        return message.content[0].text, model

    def _concrete_summarize(self, text: str, model: str) -> Dict:
        if model is None:
            model = "claude-sonnet-4-5"
        message = self.client.messages.create(
            model=model,
            max_tokens=4096,
            messages=[
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ]
        )            
        return message.content[0].text, model

claude_client = ClaudeClient(CLAUDE_API_KEY)

In [ ]:
class CohereClient(AIModelClient):
    def __init__(self, api_key):
        self.client = cohere.ClientV2(api_key=api_key)

    # https://docs.cohere.com/docs/command-a-translate
    def _concrete_translate(self, text: str, model: str) -> Dict:
        if model is None:
            model = "command-a-translate-08-2025"
        response = self.client.chat(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ],
        )
        return response.message.content[0].text, model
    
    # https://docs.cohere.com/docs/summarizing-text
    def _concrete_summarize(self, text: str, model: str) -> Dict:
        if model is None:
            model = "command-a-03-2025"
        response = self.client.chat(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ],
        )            
        return response.message.content[0].text, model

cohere_client = CohereClient(COHERE_API_KEY)

In [ ]:
class OpenAIClient(AIModelClient):
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)

    def _concrete_translate(self, text: str, model: str) -> Dict:
        if model is None:
            model  = "gpt-5-mini"

        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な翻訳者です。英語を自然で流暢な日本語に翻訳してください。"
                },
                {
                    "role": "user",
                    "content": translation_query(text)
                }
            ],
            temperature=0.3
        )        
        return response.choices[0].message.content, model

    def _concrete_summarize(self, text: str, model: str) -> Dict:
        if model is None:
            model  = "gpt-5-mini"

        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "あなたは優秀な編集者です。自然で流暢な日本語に要約してください。"
                },
                {
                    "role": "user",
                    "content": summarization_query(text)
                }
            ],
            temperature=0.3
        )            
        return response.choices[0].message.content, model

openai_client = OpenAIClient(OPENAI_API_KEY)

In [ ]:
# Project Gutenberg, https://www.gutenberg.org/ebooks/12096
TEXT_URL = 'https://www.gutenberg.org/cache/epub/12096/pg12096.txt'
TEXT_FILE = 'data/I.Nitobe_Bushido.txt'
if not os.path.exists(TEXT_FILE):
    res = requests.get(TEXT_URL)
    with open(TEXT_FILE, "w") as f:
        f.write(res.text)
    
with open(TEXT_FILE) as f:
    text = f.read()

print(f"full text size: {len(text)}")

text = text[:2000] # Truncate text for test

In [ ]:
print(json.dumps(claude_client.summarize(text), indent=2, ensure_ascii=False))
print(json.dumps(cohere_client.summarize(text), indent=2, ensure_ascii=False))


※この Notebook は、API の使い方を試す目的のものです。 AI の出力は不正確な場合があります。※

In [ ]:
print(openai_client.summarize(text))